<a href="https://colab.research.google.com/github/kahtiriya/bhavika/blob/main/frauddetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Group ID: 27**
1.   Student 1: **Bhavika Kathiriya** and **u2555134**
2.   Student 2: **Nirav Saileshbhai Umaretiya** and **u2613177**
3.   Student 3: **Anufi Prasad** and **u2564175**

---
#####Module leader: **Dr Amin Karami** & **Nadeem Qazi**


# **Initiate and Configure Spark**

In [ ]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=42c9011e9ebe660dd38cb85766847c9cca26bfe020403eed3103b7ede85dc252
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
# Initiate SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('BankApp').getOrCreate()


# **Task 1: Data Loading and Preprocessing**


### Data Explanation:

Here, we choose **Bank Account Fraud** dataset.

It comprises a total of 6 different synthetic bank account fraud tabular datasets and datasets published at **NeurIPS 2022**.

BAF is a realistic, complete, and robust test bed to evaluate novel and existing methods in ML and fair ML, and the first of its kind!

In [ ]:
# Google drive mount here
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Bank Data is load from Drive

df = spark.read.load("/content/drive/MyDrive/combined.csv", format="csv", header= True, inferschema=True)

In [ ]:
# Here we Print Bank Datasets

df.show()

+----------+-------------+-------+--------------+----------+-------------------+---------------+------------------+
|      date|txVolume(USD)|txCount|marketcap(USD)|price(USD)|exchangeVolume(USD)|activeAddresses|medianTxValue(USD)|
+----------+-------------+-------+--------------+----------+-------------------+---------------+------------------+
|2018-08-12|5324506.83294|    284|    1116316004|     11.69|           23290300|            212|     283.703661356|
|2018-08-11|8842440.96721|    480|    1156716831|     12.11|           29253100|            321|     349.972279784|
|2018-08-10| 5336434.1335|    394|    1219512788|     12.77|           37026400|            294|     240.996396601|
|2018-08-09|11809334.8381|    365|    1161589120|     12.16|           41389000|            251|     269.626930125|
|2018-08-08|8283783.58688|    881|    1253974051|     13.13|           44898500|            262|         6562.6366|
|2018-08-07| 7026460.3368|    780|    1304555493|     13.66|           4

In [ ]:
df.groupBy("farud_bool").count().show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `farud_bool` cannot be resolved. Did you mean one of the following? [`CUST_ID`, `BALANCE`, `PAYMENTS`, `TENURE`, `PURCHASES`].;
'Aggregate ['farud_bool], ['farud_bool, count(1) AS count#184L]
+- Relation [CUST_ID#17,BALANCE#18,BALANCE_FREQUENCY#19,PURCHASES#20,ONEOFF_PURCHASES#21,INSTALLMENTS_PURCHASES#22,CASH_ADVANCE#23,PURCHASES_FREQUENCY#24,ONEOFF_PURCHASES_FREQUENCY#25,PURCHASES_INSTALLMENTS_FREQUENCY#26,CASH_ADVANCE_FREQUENCY#27,CASH_ADVANCE_TRX#28,PURCHASES_TRX#29,CREDIT_LIMIT#30,PAYMENTS#31,MINIMUM_PAYMENTS#32,PRC_FULL_PAYMENT#33,TENURE#34] csv


In [ ]:
# Data Preprocessing
from pyspark.ml.feature import Imputer

# Handling Missing Values
imputer = Imputer(inputCols=["fraud_bool"], outputCols=["fraud_bool"])
data = imputer.fit(df).transform(df)

IllegalArgumentException: fraud_bool does not exist. Available: CUST_ID, BALANCE, BALANCE_FREQUENCY, PURCHASES, ONEOFF_PURCHASES, INSTALLMENTS_PURCHASES, CASH_ADVANCE, PURCHASES_FREQUENCY, ONEOFF_PURCHASES_FREQUENCY, PURCHASES_INSTALLMENTS_FREQUENCY, CASH_ADVANCE_FREQUENCY, CASH_ADVANCE_TRX, PURCHASES_TRX, CREDIT_LIMIT, PAYMENTS, MINIMUM_PAYMENTS, PRC_FULL_PAYMENT, TENURE

In [ ]:
data.show()

In [ ]:
# Import VectorAssembler
from pyspark.ml.feature import VectorAssembler

# Here, we select some column as input column and transform featues column into single vector
inputCols= ["income","name_email_similarity","days_since_request","intended_balcon_amount","velocity_6h","session_length_in_minutes"]
vecAss = VectorAssembler(inputCols = inputCols, outputCol = "features")

# Display all column in datasets and vector assember features column.
finalData = vecAss.transform(df).select('fraud_bool',"income","name_email_similarity","days_since_request","intended_balcon_amount","velocity_6h","session_length_in_minutes",'payment_type','features')
finalData.show()



In [ ]:
# Here, We are import library for RandomForestClassifier,MulticlassClassificationEvaluator and StandardScaler.
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler

# use of the VectorAssembler() function to transform our data into vector of features column.
rfFinalData = vecAss.transform(df)

# use of StandardScaler() function to scale our data.
stanardscalar=StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
data=stanardscalar.fit(rfFinalData).transform(rfFinalData)

In [ ]:
# Display Scaled_features and fraud_bool (dependent variable).
assembledData = data.select("Scaled_features","fraud_bool")
assembledData.show()


In [ ]:
# use of randomSplit() function to data split into train data and test data.
rfTrain, rfTest = assembledData.randomSplit([0.7,0.3])

In [ ]:
# Initialize RandomForestClassifier() function with parameter fraud_bool and Scaled_features.
randomForestClassifier = RandomForestClassifier(labelCol="fraud_bool",featuresCol="Scaled_features",numTrees=40)

In [ ]:
# Generate model using train data.
model = randomForestClassifier.fit(rfTrain)

In [ ]:
# Here prediction using rf_test data.
predictionTest = model.transform(rfTest)

In [ ]:
# Select (prediction, true label) and compute test error.
evaluator = MulticlassClassificationEvaluator( labelCol="fraud_bool", predictionCol="prediction", metricName="accuracy")

# **Task 3 - Model Parameter Tuning**

In [ ]:
# Import library of CrossValidator and ParamGridBuilder.
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Define parameter grid.
paramGrid = ParamGridBuilder() \
    .addGrid(randomForestClassifier.numTrees, [10, 20, 30]) \
    .addGrid(randomForestClassifier.maxDepth, [5, 10, 15]) \
    .build()

# Define CrossValidator with Random Forest, parameter grid, and evaluator.
crossval = CrossValidator(estimator=randomForestClassifier,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)

In [ ]:
# Run cross-validation, and choose the best set of parameters
cvModel = crossval.fit(data.limit(15))

In [ ]:
# Here prediction for best results.
prediction = cvModel.transform(data.limit(500))

In [ ]:
selected = prediction.select("*").limit(10)
for row in selected.collect():
    print(row)

In [ ]:
# find the best parameters and Model
bestModel = cvModel.bestModel
evaluator = cvModel.getEvaluator()
bestScore = max(cvModel.avgMetrics)
# Print the best parameters
print("Best Parameters:")
print("numTrees:", bestModel.getNumTrees)
print("maxDepth:", bestModel.getMaxDepth)
print("BestScore:", bestScore)

# **Task 4 - Model Evaluation and Accuracy Calculation**

In [ ]:
# Evaluate the accuracy.
accuracy_RF= evaluator.evaluate(predictionTest)

In [ ]:
print ("Accuracy:  ", round((accuracy_RF) * 100,2))
print ("Error: ", round((1.0 - accuracy_RF) * 100,2))

# **Task 5 - Results Visualization or Printing**

In [ ]:
# Import library matplotlib.pyplot and numpy for Visualization
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
paramGrid = cvModel.getEstimatorParamMaps()

In [ ]:
avgMetrics = cvModel.avgMetrics

In [ ]:
# set numTrees_values and maxDepth_values of each tree using for loop.
numTrees_values = [params[randomForestClassifier.numTrees] for params in paramGrid]
maxDepth_values = [params[randomForestClassifier.maxDepth] for params in paramGrid]


In [ ]:
# set parameter X,Y and Title.
plt.figure(figsize=(10, 6))
plt.plot(numTrees_values, avgMetrics, marker='o', linestyle='-')
plt.xlabel('numTrees')
plt.ylabel('Average Metric')
plt.title('Average Metric vs numTrees')
plt.grid(True)
plt.show()

# using different method with same dataset

In [ ]:
#Add Library for LinearRegression, RegressionEvaluator, Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

In [ ]:
# Set Train and Test Data
LTrainData, LTestData = assembledData.randomSplit([0.7, 0.3], seed=42)

In [ ]:
# Init LinearRegression
lr = LinearRegression(labelCol="fraud_bool",featuresCol="Scaled_features")

In [ ]:
# Train Model
pipeline = Pipeline(stages=[lr])
LModel = pipeline.fit(LTrainData)

In [ ]:
# Transform Test Data and get Predictions
Lpredictions = LModel.transform(LTestData)

# **Task 3 - Model Parameter Tuning**

In [ ]:
# Define the parameter grid
param_grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .build()

In [ ]:
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=5)

In [ ]:
# Perform cross-validation
cv_model = crossval.fit(data.limit(15))

In [ ]:
# Best model from cross-validation
best_model = cv_model.bestModel

In [ ]:
# Print the best regularization parameter
print("Best regularization parameter:", best_model._java_obj.getRegParam())

# **Task 4 - Model Evaluation and Accuracy Calculation**

In [ ]:
evaluator = RegressionEvaluator(labelCol="fraud_bool", predictionCol="prediction",metricName="rmse")
rmse = evaluator.evaluate(Lpredictions)
print("Root Mean Squared Error (RMSE):", rmse)

# **Task 5 - Results Visualization or Printing**

In [ ]:
true_values = Lpredictions.select('fraud_bool').rdd.flatMap(lambda x: x).collect()

In [ ]:
predicted_values = Lpredictions.select('prediction').rdd.flatMap(lambda x: x).collect()

In [ ]:
# Plot true vs predicted values
plt.scatter(true_values, predicted_values, alpha=0.5)
plt.plot(true_values, true_values, color='red', linestyle='--')  # Plotting y = x line for reference
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('True vs Predicted Values (Linear Regression)')
plt.show()

# 3rd different method with same dataset  Model Selection and Implementation**


In [ ]:
# Import Library ClusteringEvaluator and KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans

In [ ]:
# init Kmeans and Train Modal
kmeans = KMeans()
CModel = kmeans.fit(rfFinalData)

In [ ]:
# Define the evaluator
evaluator = ClusteringEvaluator(metricName="silhouette")

In [ ]:
# Get the best model from cross-validation
best_model = cv_model.bestModel


# **Task 3 - Model Parameter Tuning**

In [ ]:
# Define the parameter grid
param_grid = ParamGridBuilder().addGrid(kmeans.k, range(2, 11)).addGrid(kmeans.maxIter, [10, 20, 30]).build()

In [ ]:
# Define cross-validation
crossval = CrossValidator(estimator=kmeans,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=5)

In [ ]:
# Fit the cross-validator
cv_model = crossval.fit(data.limit(30000))

In [ ]:
# Get the best model from cross-validation
best_model = cv_model.bestModel

In [ ]:
# Print the best parameters
print("Best number of clusters (k):", best_model.getOrDefault("k"))
print("Best max iterations:", best_model.getOrDefault("maxIter"))

# **Task 4 - Model Evaluation and Accuracy Calculation**

In [ ]:
# Evaluate clustering performance on the test data
CPredictions = best_model.transform(data)
silhouette_score = evaluator.evaluate(CPredictions)
print("Silhouette score:", silhouette_score)

# **Task 5 - Results Visualization or Printing**

In [ ]:
# Get prediction with all cluster
cluster_labels = CPredictions.select('prediction').rdd.flatMap(lambda x: x).collect()

In [ ]:
# Select fraud_bool and prediction column
feature_cols = ["fraud_bool", "prediction"]

In [ ]:
# Extract feature values
feature_values = CPredictions.select(feature_cols[0], feature_cols[1]).collect()
feature_values = np.array(feature_values)

In [ ]:
# Plot clusters
plt.scatter(feature_values[:, 0], feature_values[:, 1], c=cluster_labels, cmap='viridis', alpha=0.5)
plt.xlabel(feature_cols[0])
plt.ylabel(feature_cols[1])
plt.title('Clustering Results (KMeans)')
plt.colorbar(label='Cluster Label')
plt.show()

---

# **Task 7 - Report HTML template**

---



In [ ]:
# install nbconvert (if facing the conversion error)
!pip3 install nbconvert

In [ ]:
# convert ipynb to html and submit this HTML file

!jupyter nbconvert --to html /content/drive/MyDrive/Group_27_CRWK_CN7030.ipynb